In [ ]:
from src.inference import predict_emotion_on_image, load_class_map
from src.evaluate import plot_confusion_matrix, print_classification_report
import json, os 

In [ ]:
# Single image demo — prints result and shows bounding box if any
IMG_PATH = "examples/face1.png"   # change to any example image you have
MODEL_PATH = "models/emotion_recognition_model.h5"
CLASSMAP_PATH = "models/class_indices.json"

if not os.path.exists(MODEL_PATH):
    print("Model not found at", MODEL_PATH, " — put your trained .h5 there or run the training cell.")
else:
    results = predict_emotion_on_image(IMG_PATH, model_path=MODEL_PATH, classmap_path=CLASSMAP_PATH, show=True)
    if not results:
        print("No faces detected in image.")
    else:
        for r in results:
            print(f"{r['label']} ({r['confidence']*100:.2f}%)  box={r['box']}")


In [ ]:
from tensorflow.keras.models import load_model
from src.dataset import create_generators
import numpy as np
from sklearn.metrics import confusion_matrix

VALID_DIR = "dataset/train"   # flow_from_directory with subset="validation" will be used
IMG_SIZE = (224,224)
BATCH_SIZE = 32

if os.path.exists(MODEL_PATH):
    # use same generator logic to get validation generator
    train_gen, val_gen = create_generators(VALID_DIR, img_size=IMG_SIZE, batch_size=BATCH_SIZE)
    model = load_model(MODEL_PATH)

    # Predict on entire validation set (may take time)
    val_steps = val_gen.samples // val_gen.batch_size + (1 if val_gen.samples % val_gen.batch_size != 0 else 0)
    preds = model.predict(val_gen, steps=val_steps, verbose=1)
    y_true = val_gen.classes  # ground-truth labels (integers)
    y_pred = np.argmax(preds, axis=1)

    labels = [k for k, v in sorted(val_gen.class_indices.items(), key=lambda item: item[1])]
    print("Labels:", labels)

    # Save confusion matrix plot
    plot_confusion_matrix(y_true, y_pred, labels, out_path="docs/confusion_matrix.png")
    from IPython.display import Image, display
    display(Image("docs/confusion_matrix.png"))
    print("\nClassification Report:\n")
    print_classification_report(y_true, y_pred, labels)
else:
    print("Model not found — run training first or copy weights to models/emotion_recognition_model.h5")
